# TP1 
## Estruturas Criptográficas - Criptografia e Segurança da Informação

### Exercício 2

2. Use o “package” Cryptography para
    1. Implementar uma AEAD com “Tweakable Block Ciphers” conforme está descrito na última secção do texto [+Capítulo 1: Primitivas Criptográficas Básicas](https://paper.dropbox.com/doc/Capitulo-1-Primitivas-Criptograficas-Basicas-YAcE9VWuF88R2fmPyvKlx#:uid=971079522289346670472132&h2=AEAD-com-%E2%80%9CTweakable-Block-Ciph).  A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-256 ou o ChaCha20.
    2. Use esta cifra para construir um canal privado de informação assíncrona com acordo de chaves feito com “X448 key exchange” e “Ed448 Signing&Verification” para autenticação  dos agentes. Deve incluir uma fase de confirmação da chave acordada.


##### Instalar packages necessários

In [182]:
%pip install cryptography

Note: you may need to restart the kernel to use updated packages.


##### Resumo 
Divisão em duas partes: 

1. a implementação da AEAD com Tweakable Block Ciphers usando o pacote "Cryptography"
2. construção do canal privado de informação assíncrona com acordo de chaves usando "X448 key exchange" e "Ed448 Signing&Verification".

#### Imports

In [183]:
import os
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.x448 import X448PrivateKey
from cryptography.hazmat.primitives.asymmetric.ed448 import Ed448PrivateKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
from datetime import datetime
import sys
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [184]:
def gen_tweaks(number_of_blocks, plaintext_length, nonce):
    cipher_tweaks = []
    # cipher tweaks [nonce|counter|0]
    for i in range(0, number_of_blocks):
        tweak = nonce + int(i).to_bytes(16, byteorder='big')
        tweak = int.from_bytes(tweak, byteorder='big')

        # remove last bit and add the final bit 0
        tweak = tweak >> 1
        tweak = tweak << 1

        tweak = tweak.to_bytes(32, byteorder='big')
        cipher_tweaks.append(tweak)
    
    # authentication tweak [nonce|plaintext_length|0]
    auth_tweak = nonce + plaintext_length.to_bytes(16, byteorder='big')
    auth_tweak = int.from_bytes(auth_tweak, byteorder='big')

    # last bit of auth_tweak to 1
    mask = 0b1
    auth_tweak = auth_tweak | mask
    auth_tweak = auth_tweak.to_bytes(32, byteorder='big')

    return cipher_tweaks, auth_tweak


In [185]:
def get_ciphertext(cipher_key, nonce, tweak, plaintext, ad):

    chacha = ChaCha20Poly1305(cipher_key)
    ciphertext = chacha.encrypt(nonce, plaintext, ad)

    xored = b''

    for (a,b) in zip(tweak, ciphertext):
        xored += bytes([a^b])

    return xored

In [186]:


def decrypt(ciphertext, tag, nonce, nonce_tweak, cipher_key, ad):
    # divide plaintext into blocks
    blocks = [ciphertext[i:i+32] for i in range(0, len(ciphertext), 32)]

    number_of_blocks = len(blocks)
    n = len(ciphertext)
    r = len(tag)
    length = n - (32 - r)

    # generate tweaks
    cipher_tweaks, auth_tweak = gen_tweaks(number_of_blocks, length, nonce_tweak)

    decrypted_blocks = []

    # decrypt blocks
    for w in range(0, number_of_blocks):
        plaintext = get_ciphertext(cipher_key, nonce, cipher_tweaks[w], blocks[w], ad)
        decrypted_blocks.append(plaintext)
    
    # authentication phase
    auth = decrypted_blocks[0]
    for i in range(1, number_of_blocks):
        xored = [(a^b).to_bytes(1,byteorder='big') for (a,b) in zip(auth, decrypted_blocks[i])]
        auth = b"".join(xored)

    generated_tag = get_ciphertext(cipher_key, nonce, auth_tweak, auth, ad)[:-1]

    # verify authentication
    if tag == generated_tag:
        decrypted_blocks[number_of_blocks - 1] = decrypted_blocks[number_of_blocks - 1][:-1]
        plaintext = b"".join(decrypted_blocks)

    else :
        return "ERROR! Different tag used in authentication."
        
    return plaintext.decode('utf-8')


In [187]:
def encrypt(plaintext, cipher_key, ad):
    # divide plaintext into blocks
    blocks = []
    for i in range(0, len(plaintext), 32):
        block = plaintext[i:i+32].encode('utf-8')
        # padding
        r = len(block)
        if r < 32:
            blocks.append(block + ((32 - r) * b'\0'))
        else:
            blocks.append(block)
    
    length = len(plaintext)
    number_of_blocks = len(blocks)

    # generate tweaks
    nonce_tweak = os.urandom(16)
    cipher_tweaks, auth_tweak = gen_tweaks(number_of_blocks, length, nonce_tweak)

    encrypted_blocks = []

    nonce = os.urandom(12)
    
    # encrypt first m-1 blocks
    for w in range(0, number_of_blocks - 1):
        ciphertext = get_ciphertext(cipher_key, nonce, cipher_tweaks[w], blocks[w], ad)
        encrypted_blocks.append(ciphertext)
        
    # encrypt last block
    r_in_bytes = int(r).to_bytes(32, byteorder='big')
    ct = get_ciphertext(cipher_key, nonce, cipher_tweaks[number_of_blocks-1], r_in_bytes, ad)

    xored = [(a^b).to_bytes(1,byteorder='big') for (a,b) in zip(ct, blocks[number_of_blocks-1])]
    last_ciphertext = b"".join(xored)
    
    encrypted_blocks.append(last_ciphertext)

    # authentication phase
    auth = blocks[0]
    for i in range(1, number_of_blocks):
        xored = [(a^b).to_bytes(1,byteorder='big') for (a,b) in zip(auth, blocks[i])]
        auth = b"".join(xored)

    tag = get_ciphertext(cipher_key, nonce, auth_tweak, auth, ad)[:r]


    # join all encrypted blocks
    ciphertext = b"".join(encrypted_blocks)

    return ciphertext, tag, nonce, nonce_tweak

In [188]:
class channel:
    def __init__(self, queue):
        self.queue = queue

    async def gen_keys(self):
        
        self.priv_Ed448_key = Ed448PrivateKey.generate()
        self.pub_Ed448_key = self.priv_Ed448_key.public_key()
        
        self.priv_x448_key = X448PrivateKey.generate()
        self.pub_x448_key = self.priv_x448_key.public_key()


    async def share_keys(self):
        
        await self.queue.put(self.pub_Ed448_key)
        
        sigEd448 = self.priv_Ed448_key.sign(
            self.pub_Ed448_key.public_bytes(Encoding.Raw, PublicFormat.Raw)
        )
        
        await self.queue.put(sigEd448)

        
        await self.queue.put(self.pub_x448_key)
        
        sigx448 = self.priv_Ed448_key.sign(
            self.pub_x448_key.public_bytes(Encoding.Raw, PublicFormat.Raw)
        )
        
        await self.queue.put(sigx448)



    async def receive_keys(self):
        
        peer_pub_Ed448_key = await self.queue.get()
        peer_pub_Ed448_key_signature = await self.queue.get()
        
        
        peer_pub_Ed448_key.verify(peer_pub_Ed448_key_signature, peer_pub_Ed448_key.public_bytes(Encoding.Raw, PublicFormat.Raw))
        self.peer_verify_key = peer_pub_Ed448_key
        
        
        peer_pub_x448_key = await self.queue.get()
        peer_pub_x448_key_signature = await self.queue.get()
        
        self.peer_verify_key.verify(peer_pub_x448_key_signature, peer_pub_x448_key.public_bytes(Encoding.Raw, PublicFormat.Raw))
        
        
        shared_key = self.priv_x448_key.exchange(peer_pub_x448_key)
        derived_key = HKDF(
            algorithm = hashes.SHA256(),
            length = 32, 
            salt = None,
            info = b"handshake data",
        ).derive(shared_key)
        self.agreed_key = derived_key

    async def send(self, plaintext):
        ad = str(datetime.now()).encode('utf-8')

        key = self.agreed_key
                
        print("Plaintext Sent: "+str(plaintext))

        
        ciphertext,tag,nounce,tweak = encrypt(plaintext, key, ad)
        
        
        
        print("\tCiphertext Sent: "+str(ciphertext))
        
        # tuplo 

        await self.queue.put(self.priv_Ed448_key.sign(ad))
        await self.queue.put(ad)

        await self.queue.put(self.priv_Ed448_key.sign(ciphertext))
        await self.queue.put(ciphertext)
        
        await self.queue.put(self.priv_Ed448_key.sign(tag))
        await self.queue.put(tag)
        
        await self.queue.put(self.priv_Ed448_key.sign(nounce))
        await self.queue.put(nounce)

        await self.queue.put(self.priv_Ed448_key.sign(tweak))
        await self.queue.put(tweak)
        


    async def receive(self):
        key = self.agreed_key

        adsig = await self.queue.get()
        ad = await self.queue.get()
        self.peer_verify_key.verify(adsig, ad)
        
        sig_ctext = await self.queue.get()
        ciphertext = await self.queue.get()
        self.peer_verify_key.verify(sig_ctext, ciphertext)

        
        sig_tag = await self.queue.get()
        tag = await self.queue.get()
        self.peer_verify_key.verify(sig_tag, tag)
        
        sig_nounce = await self.queue.get()
        nounce = await self.queue.get()
        self.peer_verify_key.verify(sig_nounce, nounce)


        
        
        sig_tweak = await self.queue.get()
        tweak = await self.queue.get()
        self.peer_verify_key.verify(sig_tweak, tweak)
        
        print("\tCiphertext Received: "+str(ciphertext))
        # decrypt(ciphertext, tag, nonce, nonce_tweak, cipher_key, ad
        plaintext = decrypt(ciphertext, tag, nounce, tweak, key, ad)
        
        
        
        print("Decrypted: "+str(plaintext)+"\n")

    async def print_agreed_key(self):
        print(self.agreed_key)


In [189]:
async def main():
    
    queue = asyncio.Queue()

    
    emissor = channel(queue)
    receptor = channel(queue)

    
    await emissor.gen_keys()
    await receptor.gen_keys()

    
    await emissor.share_keys()
    await receptor.receive_keys()
    await receptor.share_keys()
    await emissor.receive_keys()

    
    
    

    if (emissor.agreed_key == receptor.agreed_key):
        print("Chave acordada: " + str(emissor.agreed_key) + "\n")
    else : 
        print(f"Chave não foi acordada\nChave emissor: {str(emissor.agreed_key)}\nChave recetor: {str(receptor.agreed_key)}")
        sys.exit("Chave não foi acordada")

    
    
    await emissor.send("Brave Sir Robin ran away. Bravely ran away away. . .")
    await receptor.receive()
    await receptor.send("1.5")
    await emissor.receive()

asyncio.run(main())




Chave acordada: b'\x98\x80\x08;qM\x0f\xe3Ez\xce\xbb\xef9\x9a\xa5\tlA\x91\xc6O\x0c]\xd5\xe1\xcb?\x0f\xdc?\x83'

Plaintext Sent: Brave Sir Robin ran away. Bravely ran away away. . .
	Ciphertext Sent: b',\xcfO`a^\x11\xadA\xf6\xbc\xf8\xb1\xd7\xdaU\xeb\x0f\xe8E{\xf2rK):.\x1b%P\xfdg\x17\x9d\\wj^#\xb3R\xaf\xce\xf6\xa4\xdf\xcd[\xb9@\xa6K\x1a\x85\x132\x07\x1aliD&\x98\x1f'
	Ciphertext Received: b',\xcfO`a^\x11\xadA\xf6\xbc\xf8\xb1\xd7\xdaU\xeb\x0f\xe8E{\xf2rK):.\x1b%P\xfdg\x17\x9d\\wj^#\xb3R\xaf\xce\xf6\xa4\xdf\xcd[\xb9@\xa6K\x1a\x85\x132\x07\x1aliD&\x98\x1f'
Decrypted: ERROR! Different tag used in authentication.

Plaintext Sent: 1.5
	Ciphertext Sent: b'\xa4J\xa5WGO\xa5\xf4W\x03\x0b\xc8y\xa4K%\xea3\xe2fdc\xb0\x88\xd0\xaaU\xf2\xfc\xac-C'
	Ciphertext Received: b'\xa4J\xa5WGO\xa5\xf4W\x03\x0b\xc8y\xa4K%\xea3\xe2fdc\xb0\x88\xd0\xaaU\xf2\xfc\xac-C'
Decrypted: ERROR! Different tag used in authentication.



## Testes

## CRIAR TESTES PARA ADICIONAR AQUI !!!

# NOTAS FINAIS : 

MUDAR ISTO PARA CYTHON

Multiprocessing

Ver trabalhos anos anteriores

Fazer testes

Explicar melhor os packages e kernel do miniconda neste arquivo

Explicar os 2 algoritmos:

It seems like you're trying to implement an AEAD (Authenticated Encryption with Associated Data) using Tweakable Block Ciphers with either AES-256 or ChaCha20 as the primitive block cipher. You also want to use this cipher to construct a private asynchronous information channel with key agreement done using X448 key exchange and Ed448 for agent authentication, including a key confirmation phase.

